In [1]:
import pandas as pd

In [ ]:
nutrition_df = pd.read_excel('https://query.data.world/s/3yg7x3cesk6lgbtepnezfdh2m36bui')

In [2]:
df = pd.read_csv('data/data.csv')

In [4]:
df['Ingredients']

0        ['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...
1        ['2 large egg whites', '1 pound new potatoes (...
2        ['1 cup evaporated milk', '1 cup whole milk', ...
3        ['1 (¾- to 1-pound) round Italian loaf, cut in...
4        ['1 teaspoon dark brown sugar', '1 teaspoon ho...
                               ...                        
13496    ['1 cup all-purpose flour', '2/3 cup unsweeten...
13497    ['1 preserved lemon', '1 1/2 pound butternut s...
13498    ['Leftover katsuo bushi (dried bonito flakes) ...
13499    ['1 stick (1/2 cup) plus 1 tablespoon unsalted...
13500    ['12 medium to large fresh poblano chiles (2 1...
Name: Ingredients, Length: 13501, dtype: object

In [7]:
ingredients = df['Cleaned_Ingredients'].values.tolist()

In [ ]:
if 'ml' not in s and 'g' not in s
s.replace('lb.', 'lb')
s.replace('lb', 'pound')
s.replace('½', '.5')
s.replace('¾', '.75')
s.replace('cup', '128g')
s.replace('tsp.', 'tablespoon')
s.replace('tablespoon', '21.25g')
s.replace('teaspoon', '5.7g')
s.replace('pound', '453.6g')


In [9]:
from measures import Measure

In [13]:
# above might be helpful